**Connect Through SQLite Database**

In [1]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import MinMaxScaler
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [2]:
#Establish a connection
connection = sqlite3.connect('Symptoms Database.db')
# Create a cursor object
cursor = connection.cursor()

In [3]:
# Fetch data from the database
query = "SELECT * FROM ai_symptom_picker"
data = pd.read_sql_query(query, connection)
# Display the first few rows of the dataframe
print(data.head())

   gender age                                            summary  \
0    male  28  {"diseases": [], "procedures": [], "no_symptom...   
1    male  27  {"diseases": [], "procedures": [], "no_symptom...   
2  female  26  {"diseases": [], "procedures": [], "no_symptom...   
3    male  42  {"diseases": [], "procedures": [], "no_symptom...   
4  female  40  {"diseases": [], "procedures": [], "no_symptom...   

     search_term  
0    มีเสมหะ, ไอ  
1  ไอ, น้ำมูกไหล  
2        ปวดท้อง  
3      น้ำมูกไหล  
4         ตาแห้ง  


In [4]:
Symptoms = data
Symptoms

,gender,age,summary,search_term
0,male,28,"{""diseases"": [], ""procedures"": [], ""no_symptom...","มีเสมหะ, ไอ"
1,male,27,"{""diseases"": [], ""procedures"": [], ""no_symptom...","ไอ, น้ำมูกไหล"
2,female,26,"{""diseases"": [], ""procedures"": [], ""no_symptom...",ปวดท้อง
3,male,42,"{""diseases"": [], ""procedures"": [], ""no_symptom...",น้ำมูกไหล
4,female,40,"{""diseases"": [], ""procedures"": [], ""no_symptom...",ตาแห้ง
...,...,...,...,...
995,male,38,"{""diseases"": [], ""procedures"": [], ""no_symptom...","บวม, ปวดข้อ"
996,male,33,"{""diseases"": [], ""procedures"": [], ""no_symptom...",เจ็บคอ
997,male,45,"{""diseases"": [], ""procedures"": [], ""no_symptom...",ไข้
998,female,73,"{""diseases"": [], ""procedures"": [], ""no_symptom...",ไข้


In [5]:
Symptoms.head()

,gender,age,summary,search_term
0,male,28,"{""diseases"": [], ""procedures"": [], ""no_symptom...","มีเสมหะ, ไอ"
1,male,27,"{""diseases"": [], ""procedures"": [], ""no_symptom...","ไอ, น้ำมูกไหล"
2,female,26,"{""diseases"": [], ""procedures"": [], ""no_symptom...",ปวดท้อง
3,male,42,"{""diseases"": [], ""procedures"": [], ""no_symptom...",น้ำมูกไหล
4,female,40,"{""diseases"": [], ""procedures"": [], ""no_symptom...",ตาแห้ง


# **Extract Text**

In [6]:
def extract_answers(summary):
    """
    Extracts the 'answers' values from the 'yes_symptoms' list in a JSON-like string.
    """
    try:
        data = eval(summary)  # Evaluate the string as a Python dictionary
        if isinstance(data, dict) and 'yes_symptoms' in data:
            answers = []
            for symptom in data['yes_symptoms']:
                if 'answers' in symptom:
                    answers.extend(symptom['answers'])
            return ', '.join(answers)  # Join the answers with commas
        else:
            return None  # Return None if the structure is not as expected
    except (SyntaxError, TypeError, KeyError):
        return None  # Return None if there's an error parsing or accessing the data

In [7]:
def process_dataframe(df, summary_col='summary', symptoms_note_col='symptoms_note'):
    """
    Processes a DataFrame to extract 'answers' and create a new 'symptoms_note' column.
    """
    df[symptoms_note_col] = df[summary_col].apply(extract_answers)
    return df

In [8]:
# Process the DataFrame
Symptoms = process_dataframe(Symptoms)

In [9]:
Symptoms.head()

,gender,age,summary,search_term,symptoms_note
0,male,28,"{""diseases"": [], ""procedures"": [], ""no_symptom...","มีเสมหะ, ไอ","ลักษณะ เสมหะเปลี่ยนสีเหลือง/เขียว, ระยะเวลา ไม..."
1,male,27,"{""diseases"": [], ""procedures"": [], ""no_symptom...","ไอ, น้ำมูกไหล","ระยะเวลา 1-3 สัปดาห์, ลักษณะ ไอไม่มีเสมหะ ไอแห..."
2,female,26,"{""diseases"": [], ""procedures"": [], ""no_symptom...",ปวดท้อง,"บริเวณ รอบๆสะดือ, ระยะเวลา ตั้งแต่ 1 วัน ถึง 1..."
3,male,42,"{""diseases"": [], ""procedures"": [], ""no_symptom...",น้ำมูกไหล,"ระยะเวลา น้อยกว่า 10 วัน, ประวัติ ATK ยังไม่ได..."
4,female,40,"{""diseases"": [], ""procedures"": [], ""no_symptom...",ตาแห้ง,การรักษาก่อนหน้า ไม่เคย


In [10]:
Symptoms.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   gender         1000 non-null   object
 1   age            1000 non-null   object
 2   summary        1000 non-null   object
 3   search_term    1000 non-null   object
 4   symptoms_note  1000 non-null   object
dtypes: object(5)
memory usage: 39.2+ KB


# **Association Rules** For extracting relevant symptoms and match them with symptom set

In [11]:
def preprocess_symptoms(Symptoms):
    # Handle NaN values in symptoms_note
    Symptoms['symptoms_note'] = Symptoms['symptoms_note'].fillna('')
    
    # Split symptoms into list
    Symptoms['symptoms_list'] = Symptoms['symptoms_note'].str.split(',')
    
    # Get unique symptoms - filter out empty strings and None values
    all_symptoms = set()
    for symptoms in Symptoms['symptoms_list']:
        if isinstance(symptoms, list):
            # Strip whitespace and filter out empty strings
            cleaned_symptoms = [s.strip() for s in symptoms if s and s.strip()]
            all_symptoms.update(cleaned_symptoms)
    
    # Create one-hot vectors manually
    symptoms_encoded = pd.DataFrame(0, index=Symptoms.index, columns=list(all_symptoms))
    
    for idx, symptoms in enumerate(Symptoms['symptoms_list']):
        if isinstance(symptoms, list):
            for symptom in symptoms:
                # Only process non-empty symptoms
                if symptom and symptom.strip():
                    symptoms_encoded.loc[idx, symptom.strip()] = 1
    
    # Handle any remaining NaN values
    symptoms_encoded = symptoms_encoded.fillna(0)
    
    # Apply min-max scaling
    scaler = MinMaxScaler()
    symptoms_scaled = pd.DataFrame(
        scaler.fit_transform(symptoms_encoded),
        columns=symptoms_encoded.columns
    )
    
    return symptoms_scaled

In [12]:
def generate_association_rules(symptoms_encoded, min_support=0.1, min_confidence=0.5):
    # Find frequent itemsets
    frequent_itemsets = apriori(symptoms_encoded, 
                              min_support=min_support, 
                              use_colnames=True)
    
    # Generate rules
    rules = association_rules(frequent_itemsets, 
                            metric="confidence", 
                            min_threshold=min_confidence)
    
    return rules

In [13]:
def select_features(rules, symptoms_encoded, top_n=10):
    # Select features based on lift score
    important_features = rules.nlargest(top_n, 'lift')
    selected_features = set()
    
    for _, row in important_features.iterrows():
        selected_features.update(row['antecedents'])
        selected_features.update(row['consequents'])
    
    return symptoms_encoded[list(selected_features)]

In [14]:
# Main implementation
def implement_symptom_analysis(Symptoms):
    # Preprocess the data
    symptoms_encoded = preprocess_symptoms(Symptoms)
    
    # Generate association rules
    rules = generate_association_rules(symptoms_encoded)
    
    # Select important features
    selected_features = select_features(rules, symptoms_encoded)
    
    print("Shape of encoded symptoms:", symptoms_encoded.shape)
    print("\nNumber of association rules generated:", len(rules))
    print("\nTop features by lift score:", list(selected_features.columns))
    
    return rules, selected_features

In [15]:
# Example usage
rules, selected_features = implement_symptom_analysis(Symptoms)

Shape of encoded symptoms: (1000, 276)

Number of association rules generated: 9

Top features by lift score: ['ประวัติใกล้ชิดผู้ป่วยโควิด-19 ไม่ได้ใกล้ชิด', 'การรักษาก่อนหน้า ไม่เคย', 'ประวัติ ATK ยังไม่ได้ทำ']


d:\Symptoms Recommendation\.venv\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


In [16]:
def find_associated_symptoms(input_symptom, Symptoms):
    """
    Find associated symptoms for a specific symptom
    """
    # Clean and prepare the data
    symptoms_pairs = Symptoms[['symptoms_note', 'search_term']].dropna()
    
    # Find exact row matching the specific symptom
    matching_row = symptoms_pairs[
        symptoms_pairs['search_term'].str.contains(input_symptom, case=False, na=False) |
        symptoms_pairs['symptoms_note'].str.contains(input_symptom, case=False, na=False)
    ].iloc[0] if len(symptoms_pairs) > 0 else None
    
    if matching_row is not None:
        # Extract all symptoms from the matching row
        symptoms = str(matching_row['symptoms_note']).split(',')
        symptoms = [s.strip() for s in symptoms if s.strip()]
        
        # Filter out the input symptom
        associated_symptoms = [
            s for s in symptoms 
            if input_symptom.lower() not in s.lower()
        ]
        
        print(f"\nResults for '{input_symptom}':")
        print(f"Search Term: {matching_row['search_term']}")
        print("Associated symptoms:")
        for symptom in associated_symptoms:
            print(f"- {symptom}")
        print("-" * 50)
        
        return associated_symptoms
    
    return []

In [17]:
# Test each case separately
print("Case 1: ปวดท้อง")
case1 = find_associated_symptoms('ปวดท้อง', Symptoms)

Case 1: ปวดท้อง

Results for 'ปวดท้อง':
Search Term: ปวดท้อง
Associated symptoms:
- บริเวณ รอบๆสะดือ
- ระยะเวลา ตั้งแต่ 1 วัน ถึง 1 สัปดาห์
- ระดับ ปวดจนไม่สามารถทำงานได้
- การรักษาก่อนหน้า ไม่เคย
--------------------------------------------------


In [18]:
print("\nCase 2: ไอ")
case2 = find_associated_symptoms('ไอ', Symptoms)


Case 2: ไอ

Results for 'ไอ':
Search Term: มีเสมหะ, ไอ
Associated symptoms:
- ลักษณะ เสมหะเปลี่ยนสีเหลือง/เขียว
- ระยะเวลา ไม่เกิน 1 สัปดาห์ (ไม่เกิน 7 วัน)
- การรักษาก่อนหน้า ไม่เคย
--------------------------------------------------
